# Assignment #5: Extraction of subject–verb–object triples
Author: Pierre Nugues

## Objectives

In this assignment, you will extract relations from a parsed sentence involving two words or entities. You will start with pairs of words, namely a subject and its verb, and then extend your programs to triples: subject, verb, and object. In the triples, the subject and the object are the entities, and the verb represents the relation. 

$$
\text{Subject} \xrightarrow[\text{}]{\text{Verb}} \text{Object}
$$

The overall work is inspired by the _Prismatic_ knowledge base used in the IBM Watson system, where the subject, verb, and object triples are a way to extract knowledge from text.  See <a href="http://www.aclweb.org/anthology/W/W10/W10-0915.pdf">this paper</a> for details. 

You will apply the extraction to multilingual texts: 
1. First you will use a parsed corpus of Swedish; and then
2. You will apply it to other languages.
            
The objectives of this assignment are to:
* Extract the subject–verb pairs from a parsed corpus
* Extend the extraction to subject–verb–object triples
* Understand how dependency parsing can help create a knowledge base
* Write a short report of 1 to 2 pages on the assignment

## Corpus

As corpora, you will use the Universal Dependencies: https://universaldependencies.org/.
1. In the first part of the assignment, you will focus on Swedish as it is easier to understand for most students, and then 
2. Move on to all the other languages. 

You will only consider the training sets of each corpus.

### Choosing a parsed corpus

1. Download manually the latest version of Universal dependencies (2.8.1); see here `https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-3687`;
2. Alternatively, you can run the command below, preferably from a terminal window

In [2]:
!curl "https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3687/ud-treebanks-v2.8.tgz?sequence=1&isAllowed=y" --output my_ud_file.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:04 --:--:--     0^C


Uncompress the archive and:
1. Go to the Swedish _Talbanken_ corpus;
2. Read the CoNLL-U annotation here: https://universaldependencies.org/format.html

### Examining the annotation

You will carry out the following steps and describe them in your report:

1. Draw graphical representations of the two first Swedish sentences of the training set. You will include these drawings in your report;
2. Visualize these sentences with this tool: http://spyysalo.github.io/conllu.js/ and check that you have the same results;
3. Apply the dependency parser for Swedish of the <a href="http://vilde.cs.lth.se:9000/">Langforia pipelines</a> to these sentences (only the text of each sentence). You will have to select Swedish and activate both `Token` and `DependencyRelation`. Link to Lanforia pipelines: <a href="http://vilde.cs.lth.se:9000/">http://vilde.cs.lth.se:9000/</a>. You will describe possible differences.

## Programming

### Swedish

You will extract all the subject–verb pairs and the subject–verb–object triples from the Swedish _Talbanken_ training corpus. To start the program, you can use the CoNLL-U reader available in the cells below.
This program works for the other corpora. You can also program a reader yourself starting from the one you used to read the CoNLL 2000 format in the fourth lab or from scratch. 

#### Imports

In [1]:
import os

### Corpus location

Here are the corpus locations you will use. You may have to adjust `ud_path`.

In [10]:
ud_path = 'ud-treebanks-v2.8/'

In [11]:
path_sv = ud_path + 'UD_Swedish-Talbanken/sv_talbanken-ud-train.conllu'
path_fr = ud_path + 'UD_French-GSD/fr_gsd-ud-train.conllu'
path_ru = ud_path + 'UD_Russian-SynTagRus/ru_syntagrus-ud-train.conllu'
path_en = ud_path + 'UD_English-EWT/en_ewt-ud-train.conllu'

The column names of the CoNLL-U corpora

In [12]:
column_names_u = ['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC']

#### Functions to read the CoNLL-U files

In [13]:
def read_sentences(file):
    """
    Creates a list of sentences from the corpus
    Each sentence is a string
    :param file:
    :return:
    """
    f = open(file, encoding='utf-8').read().strip()
    sentences = f.split('\n\n')
    return sentences

In [14]:
def split_rows(sentences, column_names):
    """
    Creates a list of sentence where each sentence is a list of lines
    Each line is a dictionary of columns
    :param sentences:
    :param column_names:
    :return:
    """
    new_sentences = []
    root_values = ['0', 'ROOT', 'ROOT', 'ROOT', 'ROOT', 'ROOT', '0', 'ROOT', '0', 'ROOT']
    start = [dict(zip(column_names, root_values))]
    for sentence in sentences:
        rows = sentence.split('\n')
        sentence = [dict(zip(column_names, row.split('\t'))) for row in rows if row[0] != '#']
        sentence = start + sentence
        new_sentences.append(sentence)
    return new_sentences

#### Reading the corpus

We load the Swedish _Talbanken_ corpus.

In [15]:
sentences = read_sentences(path_sv)
formatted_corpus = split_rows(sentences, column_names_u)

In [16]:
sentences = read_sentences(path_sv)
formatted_corpus = split_rows(sentences, column_names_u)

In [18]:
len(formatted_corpus)

4303

The parsed sentence: _Genom skattereformen införs individuell beskattning (särbeskattning) av arbetsinkomster._

In [19]:
formatted_corpus[1]

[{'ID': '0',
  'FORM': 'ROOT',
  'LEMMA': 'ROOT',
  'UPOS': 'ROOT',
  'XPOS': 'ROOT',
  'FEATS': 'ROOT',
  'HEAD': '0',
  'DEPREL': 'ROOT',
  'DEPS': '0',
  'MISC': 'ROOT'},
 {'ID': '1',
  'FORM': 'Genom',
  'LEMMA': 'genom',
  'UPOS': 'ADP',
  'XPOS': 'PP',
  'FEATS': '_',
  'HEAD': '2',
  'DEPREL': 'case',
  'DEPS': '2:case',
  'MISC': '_'},
 {'ID': '2',
  'FORM': 'skattereformen',
  'LEMMA': 'skattereform',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|SIN|DEF|NOM',
  'FEATS': 'Case=Nom|Definite=Def|Gender=Com|Number=Sing',
  'HEAD': '3',
  'DEPREL': 'obl',
  'DEPS': '3:obl:genom',
  'MISC': '_'},
 {'ID': '3',
  'FORM': 'införs',
  'LEMMA': 'införa',
  'UPOS': 'VERB',
  'XPOS': 'VB|PRS|SFO',
  'FEATS': 'Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Pass',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': '0:root',
  'MISC': '_'},
 {'ID': '4',
  'FORM': 'individuell',
  'LEMMA': 'individuell',
  'UPOS': 'ADJ',
  'XPOS': 'JJ|POS|UTR|SIN|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Numb

#### Converting the lists in dictionaries

To ease the processing of some corpora, you will use a dictionary represention of the sentences. The keys will be the `ID` values. We do this because `ID` is not necessarily a number.

In [20]:
def convert_to_dict(formatted_corpus):
    """
    Converts each sentence from a list of words to a dictionary where the keys are id
    :param formatted_corpus:
    :return:
    """
    formatted_corpus_dict = []
    for sentence in formatted_corpus:
        sentence_dict = {}
        for word in sentence:
            sentence_dict[word['ID']] = word
        formatted_corpus_dict.append(sentence_dict)
    return formatted_corpus_dict

In [21]:
formatted_corpus_dict = convert_to_dict(formatted_corpus)
formatted_corpus_dict[1]

{'0': {'ID': '0',
  'FORM': 'ROOT',
  'LEMMA': 'ROOT',
  'UPOS': 'ROOT',
  'XPOS': 'ROOT',
  'FEATS': 'ROOT',
  'HEAD': '0',
  'DEPREL': 'ROOT',
  'DEPS': '0',
  'MISC': 'ROOT'},
 '1': {'ID': '1',
  'FORM': 'Genom',
  'LEMMA': 'genom',
  'UPOS': 'ADP',
  'XPOS': 'PP',
  'FEATS': '_',
  'HEAD': '2',
  'DEPREL': 'case',
  'DEPS': '2:case',
  'MISC': '_'},
 '2': {'ID': '2',
  'FORM': 'skattereformen',
  'LEMMA': 'skattereform',
  'UPOS': 'NOUN',
  'XPOS': 'NN|UTR|SIN|DEF|NOM',
  'FEATS': 'Case=Nom|Definite=Def|Gender=Com|Number=Sing',
  'HEAD': '3',
  'DEPREL': 'obl',
  'DEPS': '3:obl:genom',
  'MISC': '_'},
 '3': {'ID': '3',
  'FORM': 'införs',
  'LEMMA': 'införa',
  'UPOS': 'VERB',
  'XPOS': 'VB|PRS|SFO',
  'FEATS': 'Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Pass',
  'HEAD': '0',
  'DEPREL': 'root',
  'DEPS': '0:root',
  'MISC': '_'},
 '4': {'ID': '4',
  'FORM': 'individuell',
  'LEMMA': 'individuell',
  'UPOS': 'ADJ',
  'XPOS': 'JJ|POS|UTR|SIN|IND|NOM',
  'FEATS': 'Case=Nom|Definite=Ind|D

### Extracting the subject-verb pairs

Now you will extract the subject-verb pairs, where you will set the words in lowercase. In the second sentence of the corpus, this corresponds to `(beskattning, införs)`. You will call the function `extract_pairs(formatted_corpus_dict)` and and you will store the results in a `pairs_sv` variable. All the corpora in the universal dependencies format use the same names for the grammatical functions. The subject and direct object are respectively `nsubj` and `obj`.

You can use the algorithm you want. However, here are some hints on the results:
* You will extract all the subject-verb pairs in the corpus. In the extraction, just check the function between two words. Do not check if the part of speech is a verb or a noun in the pair. You will also ignore the possible function suffixes as in `nsubj:pass`, where `pass` means passive.
* You will return the results as Python's dictionaries, where the key will be the pair and the value, the count, as for instance `{(beskattning, införs): 1}`. Be sure you understand the Python dictionaries and note that you can use tuples as keys.

In [39]:
# Write your code here
def extract_pairs(corpus):
    #Set to lowercase
    for sentences in corpus:
        for key in sentences.keys():
            if key != '0': #Skips the rubric of each set
                sentences[key]['FORM'] = sentences[key]['FORM'].lower()
                print(sentences[key]['FORM'])
tmp=extract_pairs(formatted_corpus_dict)


individuell
beskattning
av
arbetsinkomster
genom
skattereformen
införs
individuell
beskattning
(
särbeskattning
)
av
arbetsinkomster
.
det
innebär
bl.a.
att
endast
en
skatteskala
kommer
att
finnas
för
beräkning
av
statlig
inkomstskatt
.
den
blir
gemensam
för
alla
inkomsttagare
oavsett
civilstånd
.
den
gäller
även
för
oskifta
dödsbon
och
familjestiftelser
.
de
inkomster
som
på
detta
sätt
beskattas
individuellt
kallas
a-inkomster
.
dessa
inkomster
är
*
inkomst
av
tjänst
-
lön
,
pension
,
livränta
,
undantagsförmåner
och
övrig
tjänsteinkomst
.
(
undantag
:
periodiskt
understöd
eller
därmed
jämförlig
periodisk
inkomst
.
)
*
inkomst
av
jordbruksfastighet
-
om
den
skattskyldige
arbetat
i
jordbruket
i
ej
blott
ringa
omfattning
.
*
inkomst
av
rörelse
-
om
den
skattskyldige
arbetat
i
rörelsen
i
ej
blott
ringa
omfattning
.
makars
övriga
inkomster
är
b-inkomster
och
skall
som
tidigare
sambeskattas
.
sådana
b-inkomster
är
t.ex.
inkomst
av
kapital
,
tillfällig
förvärvsverksamhet
,
periodiskt
unders

räcker
yrkesbefälet
inte
till
att
fylla
mer
än
en
liten
del
av
alla
befälsbefattningar
.
90
procent
av
befälet
i
krig
är
värnpliktiga
eller
reservbefäl
.
för
att
de
skall
klara
sina
uppgifter
och
kunna
bära
det
ansvar
de
har
för
sina
soldater
måste
de
få
tillräckligt
lång
utbildning
.
de
som
tagits
ut
till
underofficersutbildning
kommer
att
få
det
största
ansvaret
och
de
svåraste
uppgifterna
av
alla
värnpliktiga
.
de
lär
sig
föra
befäl
över
en
pluton
på
omkring
40
man
.
deras
grundutbildning
tar
450
dagar
.
om
de
klarar
sig
bra
kan
de
sedan
anmäla
sig
till
officersutbildning
,
som
alltid
är
frivillig
.
de
som
tagits
ut
till
underbefälsutbildning
utbildas
till
gruppchefer
.
deras
utbildning
tar
345
dagar
.
även
för
meniga
är
utbildningstiden
olika
lång
.
värnpliktiga
kategori
e
får
uppgifter
som
det
tar
särskilt
lång
tid
att
lära
.
en
del
av
dessa
värnpliktiga
skall
t ex
handha
särskilt
komplicerad
och
dyrbar
materiel
.
de
har
därför
lika
lång
grundutbildning
som
värnpliktiga
uttagna
ti

försedda
med
postnummer
.
hur
använder
man
postnummerkatalogen
?
del
1
:
här
är
landets
alla
postanstalter
upptagna
i
bokstavsordning
,
och
här
finner
man
postnumren
för
det
stora
flertalet
orter
.
för
en
del
orter
,
som
har
flera
postanstalter
,
finns
flera
olika
postnummer
angivna
.
då
används
alltid
,
om
det
rör
sig
om
post
med
gatuadress
etc
,
det
lägsta
av
dessa
nummer
.
när
det
gäller
en
försändelse
med
fack-
eller
boxadress
använder
man
postnumret
för
den
av
ortens
postanstalter
,
där
adressaten
hämtar
sin
post
i
fack
eller
box
.
för
post
som
adresseras
poste
restante
är
postnumret
detsamma
som
för
fackadresserad
post
.
för
en
del
orter
består
upplysningen
i
del
1
enbart
av
en
hänvisning
till
en
sida
i
del
2
av
katalogen
.
om
en
adressort
inte
går
att
finna
i
del
1
,
bör
man
se
efter
om
den
förekommer
i
del
3
,
förteckning
över
indragna
och
namnändrade
postanstalter
,
som
börjar
på
sid
189
.
där
finns
i
så
fall
upplysning
om
postnummer
och
ny
adresspostanstalt
.
del
2
:
i
en
del

tror
att
ni
är
gravid
,
tänk
på
att
urinprovet
blir
tillförlitligt
först
4
veckor
efter
det
samlag
då
befruktning
kan
ha
ägt
rum
.
du
har
chansen
att
vinna
en
halv
miljon
skattefritt
i
statens
stora
lotteri
på
skatteåterbäringen
1970
.
och
du
har
inte
bara
en
chans
att
vinna
.
lotteriet
har
nämligen
två
dragningar
.
en
i
juni
då
du
kan
vinna
en
högsta
vinst
på
1/4
miljon
kronor
.
en
i
november
/
december
1971
då
vinstlistan
toppas
av
rekordvinsten
1/2
miljon
.
vid
båda
dragningarna
dessutom
massor
av
andra
vinster
,
på
100000
,
25000
,
10000
,
5000
kr
etc
.
och
alla
vinsterna
är
skattefria
.
genom
att
nu
satsa
din
skatteåterbäring
,
helt
eller
delvis
(
minst
100
kr
)
,
får
du
vara
med
i
båda
dragningarna
.
och
inom
ramen
för
det
stora
skattelotteriet
deltar
du
också
i
ett
särskilt
'
födelsedagslotteri
'
.
läs
mer
om
detta
på
andra
sidan
!
samtidigt
får
du
högsta
sparränta
plus
en
skattefri
sparpremie
.
din
insats
i
skattelotteriet
är
samtidigt
ett
sparande
som
återgår
till
dig
med
reko

22
och
se
moderskap
sida
15
.
barnavård
kommunens
barnavårdsnämnd
ska
verka
för
att
barn
och
ungdomar
får
en
gynnsam
utveckling
och
goda
uppväxtförhållanden
.
detta
innebär
bl.a.
*
att
ordna
daghem
,
lekskolor
,
barnkolonier
och
fritidsverksamhet
för
barn
och
ungdom
,
*
att
lämna
råd
och
upplysningar
om
uppfostran
till
de
föräldrar
som
begär
det
,
*
att
ordna
vård
åt
fosterbarn
,
ge
råd
och
hjälp
i
adoptionsfrågor
,
*
att
betala
ut
barnbidrag
och
bidragsförskott
,
*
att
gripa
in
och
hjälpa
om
barn
och
ungdom
råkar
i
nöd
,
utsätts
för
vanvård
eller
kommer
på
villovägar
,
*
att
i
vissa
fall
besluta
om
övervakning
eller
att
flytta
den
unge
från
hemmet
,
t.ex.
till
en
ungdomsvårdsskola
.
barnavårdsnämndens
ansvar
för
omvårdnaden
av
barn
och
av
ungdomar
varar
tills
de
fyller
20
år
(
blir
myndiga
)
.
barnavårdsnämndens
ledamöter
väljs
av
kommunal-
eller
stadsfullmäktige
.
nämndens
verksamhet
grundas
på
barnavårdslagen
.
i
större
kommuner
har
barnavårdsnämnden
anställd
personal
som
tar
emot
d

...

polisstatistiken
finns
i
statistisk
årsbok
.
se
vidare
brottsbalken
kap.
8
(
stöld
,
rån
och
andra
tillgreppsbrott
)
;
sveri
m.fl.
,
kap.
1
;
nelson
,
kap.
de
särskilda
brotten
.
varför
har
brottsligheten
-
och
speciellt
ungdomsbrottsligheten
-
i
sverige
ökat
så
starkt
under
1950-
och
1960-talet
?
se
sou
1964:58
,
kap.
3
;
blomberg
,
kap.
11
;
boalt
m.fl.
,
kap.
brottslighet
.
vilken
åldersgrupp
var
1961
den
största
inom
brottstyperna
inbrottsstöld
,
stöld
,
snatteri
,
biltillgrepp
,
rån
,
bedrägeri
,
skadegörelse
?
förklara
olikheterna
i
fördelningen
.
se
sou
1964:58
,
s.
26
.
ta
reda
på
liknande
siffror
för
1965
och
1966
.
se
brottsligheten
1965
och
1966
(
sos
)
.
se
också
sveri
m.fl.
,
kap.
1
.
vilka
möjligheter
att
förebygga
ungdomsbrottslighet
finns
eller
borde
finnas
?
se
sou
1964:58
,
kap.
4
;
blomberg
,
kap.
3
;
eriksson
,
kap.
samhället
och
ungdomsbrottslingarna
.
i
vilken
utsträckning
är
den
dolda
brottsligheten
liktydig
med
barn-
och
ungdomsbrottslighet
?
se
åkerman
,
kap.
s

återfall
i
kasuistisk
etik
som
lägger
större
vikt
vid
den
formella
situationen
än
vid
den
personliga
avgörelsen
.
kristen
etik
tillmäter
människans
egen
besinning
inför
kärleksbudet
avgörande
betydelse
.
exemplet
från
utah
ger
också
anledning
att
ställa
en
annan
fråga
.
är
det
ur
kristen
synpunkt
försvarligt
att
driva
en
sexualmoral
som
leder
till
moralisk
upplösning
för
en
mindre
grupp
samtidigt
som
majoriteten
kanske
uppfyller
de
formella
kraven
?
det
är
ett
svårt
problem
men
följande
resonemang
tycker
vi
är
rimligt
.
om
flertalets
avhållsamhet
är
påtvingad
av
en
förhärskande
social
kontroll
,
är
det
inte
längre
fråga
om
etisk
avgörelse
i
sann
d.v.s.
personlig
mening
.
vi
får
en
miljö
med
pressure
groups
mot
vilkas
normer
man
inte
vågar
bryta
,
därför
att
man
är
rädd
för
repressalier
i
den
ena
eller
andra
formen
.
ett
sådant
system
skapar
fruktan
och
överdimensionerad
skuld
hos
alla
i
denna
miljö
.
en
social
gemenskap
kan
å
andra
sidan
inte
existera
utan
former
av
social
kontroll
som

kärnfamiljens
växande
betydelse
som
primärgrupp
eftersom
samhället
saknar
alternativa
former
för
samlevnad
i
varaktiga
lojala
och
stödgivande
kollektiv
.
bortsett
från
familjen
och
i
någon
mån
arbetsgruppen
är
det
huvudsakligen
medlemskap
i
sekundära
grupper
,
massbildningar
,
organisationer
,
tillfälliga
bekantskaper
och
anonyma
församlingar
som
karakteriserar
individens
kontakter
och
samvaro
med
andra
.
i
den
familjesociologiska
litteraturen
möter
man
kontroversiella
uppfattningar
om
den
lilla
,
relativt
isolerade
och
i
teknisk-ekonomisk
mening
irrationellt
organiserade
familjegruppens
inflytande
på
sina
medlemmar
.
vid
en
analys
av
familjens
primärgruppsfunktion
har
man
att
skilja
på
två
problemkomplex
:
å
ena
sidan
familjemedlemmarnas
inre
relationer
och
å
andra
sidan
familjemedlemmarnas
relationer
utåt
.
i
det
första
fallet
kan
man
diskutera
i
vad
mån
det
är
en
realistisk
föreställning
att
kärnfamiljen
mäktar
med
huvudansvaret
-
för
att
inte
säga
monopolet
-
att
tillfredsställa
sa

och
'
världsligt
'
vara
nära
sammanfogat
.
naturligtvis
har
tanken
på
makarnas
gemensamma
ansvar
för
barnen
varit
en
betydande
faktor
,
när
kyrkan
slagit
vakt
om
äktenskapet
.
men
äktenskapets
värde
kan
inte
begränsas
till
detta
,
det
vore
att
ge
en
pragmatisk
motivering
som
vore
alldeles
för
ytlig
.
även
om
vi
tänker
oss
någon
sorts
social
utopi
,
där
samhället
övertagit
hela
ansvaret
för
avkomman
-
och
den
utopin
förespeglade
t.ex.
någon
debattör
i
trolöshetsdebatten
för
en
del
år
sen
-
så
skulle
detta
inte
upphäva
äktenskapets
ställning
i
kristen
etik
.
vi
har
här
en
sorts
parallell
till
vad
vi
tidigare
sade
om
preventivmedlens
betydelse
för
en
eventuell
omvärdering
av
sexuellt
samliv
.
det
är
överhuvud
aldrig
självklart
i
etiken
vilka
nya
värderingar
som
kan
följa
från
ett
förändrat
faktaläge
,
och
i
bägge
dessa
frågor
finns
det
anledning
att
undvika
en
förenklad
tolkning
av
den
kristna
ståndpunkten
.
vad
konstituerar
äktenskapet
?
vad
är
det
nu
som
konstituerar
äktenskapet
?
det
ä

fall
och
tar
till
moraliserande
brösttoner
.
de
som
talar
'
för
'
hemmamamman
beskriver
daghems-
och
nyckelbarnen
som
de
presumtiva
kunderna
i
våra
fängelser
.
de
som
försvarar
de
förvärvsarbetande
mödrarna
ser
i
stället
dessa
kvinnors
barn
som
mer
ansvarsfulla
,
lyckligare
och
självständigare
än
hemmamammans
,
som
blir
överbeskyddade
och
osäkra
.
också
bland
barnpsykiater
,
barnläkare
och
barnpsykologer
har
skadeverkningarna
av
att
skilja
småbarn
från
deras
mor
för
längre
eller
kortare
tid
varit
föremål
för
livlig
forskning
och
debatt
,
utan
att
man
hittills
kommit
fram
till
klara
resultat
.
år
1951
gjorde
den
engelske
läkaren
j.
bowlby
för
who:s
räkning
en
sammanställning
av
dittills
gjorda
undersökningar
över
hur
späda
och
små
barn
,
som
för
någon
tid
helt
skilts
från
sin
mor
,
utvecklas
.
resultatet
var
skrämmande
.
det
visade
sig
att
dessa
barn
kom
att
sakna
förmåga
till
verklig
känsloanknytning
,
de
blev
oförmögna
till
inlevelse
i
andra
människor
,
till
att
verkligen
tycka
om
någ

In [14]:
pairs_sv = extract_pairs(formatted_corpus_dict)

You will compute the total number of subject-verb pairs. You should find 6,083 pairs.

In [15]:
sum([pairs_sv[pair] for pair in pairs_sv])

6083

#### Finding the most frequent pairs

You will sort your pairs by frequency and by lexical order of the pairs and store the five most frequent pairs in the `freq_pairs_sv` variable as in:
```
freq_pairs_sv = [(('som', 'har'), 45),

 ...]
 ````

Here are the frequencies you should find:
```
45
19
19
```
When pairs have equal frequencies, you will sort them by alphabetical order, first term, and then second term of the pair.

In all the experiments, we will keep the `nbest` most frequent. In the first experiments, we set `nbest` to 3 first. We will set it to 5 in the last experiment.

In [16]:
nbest = 3

In [17]:
sorted_pairs = sorted(pairs_sv, key=lambda x: (-pairs_sv[x], x))
freq_pairs_sv = [(pair, pairs_sv[pair]) for pair in sorted_pairs][:nbest]

In [ ]:
freq_pairs_sv

### Extracting the subject-verb-object triples

You will now extract all the subject–verb–object triples of the corpus. The object function uses the `obj` code.

In [19]:
# Write your code here

In [20]:
triples_sv = extract_triples(formatted_corpus_dict)

Compute the total number of triples. You should find 2054 triples.

In [21]:
sum([triples_sv[triple] for triple in triples_sv])

2054

#### Finding the most frequent triples

You will sort your triples by frequency and by lexical order of the pairs and store the three most frequent triples in the `freq_triples_sv` variable as in:
```
freq_triples_sv = [(('man', 'vänder', 'sig'), 14),

 ...]
 ````

Here are the frequencies you should find:
```
14
5
3
```
As with the pairs of equal frequencies, you will sort the triples by alphabetical order.

In [22]:
sorted_triples = sorted(triples_sv, key=lambda x: (-triples_sv[x], x))
freq_triples_sv = [(triple, triples_sv[triple]) for triple in sorted_triples][:nbest]

In [ ]:
freq_triples_sv

### Multilingual Corpora

Once your program is working on Swedish, you will apply it to all the other languages in universal dependencies. The code below returns all the files from a folder with a suffix. Here we consider the training files only.

In [24]:
def get_files(dir, suffix):
    """
    Returns all the files in a folder ending with suffix
    Recursive version
    :param dir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(dir):
        path = dir + '/' + file
        if os.path.isdir(path):
            files += get_files(path, suffix)
        elif os.path.isfile(path) and file.endswith(suffix):
            files.append(path)
    return files

In [25]:
files = get_files(ud_path, 'train.conllu')

#### Dealing with the indices

Some corpora expand some tokens into multiwords. This is the case in French, Spanish, German, as well as English in some corpora.
        The table below shows examples of such expansions.
        <table style="width:100%">
            <tr>
                <th>French</th>
                <th>Spanish</th>
                <th>German</th>
            </tr>
            <tr>
                <td><i>du</i>: de le
                </td>
                <td><i>del</i>: de el
                </td>
                <td><i>zur</i>: zu der
                </td>
            </tr>
            <tr>
                <td><i>des</i>: de les
                </td>
                <td><i>vámonos</i>: vamos nos
                </td>
                <td><i>im</i>: in dem
                </td>
            </tr>
        </table>
        In the corpora, you have the original tokens as well as the multiwords as with <i>vámonos al mar</i> as in:
        <pre>
1-2 vámonos _
1 vamos ir
2 nos nosotros
3-4 al _
3 a a
4 el el
5 mar mar
</pre>Read the format description for the details: [<a
                href="http://universaldependencies.org/format.html">CoNLL-U format</a>].

If you represent the sentences as lists, the item indices are not reliable: In the format description,
        the token at position 1 is <i>vamos</i> and not <i>vámonos</i>.
        You have two ways to cope with this:
1. Either remove all the lines that include a range in the `ID` field, or
2. Encode the sentences as dictionaries (I felt this was preferable), where the keys are the `ID` numbers. This is what `convert_to_dict()` does. Here are the results for a sentence from the French CoNLL-U corpus:
_Les iris du mâles sont jaunes toute l'année._ Note the `3-4` index and it expansion in `3`and `4`:
```
{'0': {'ID': '0',  'FORM': 'ROOT',  'LEMMA': 'ROOT',  'UPOS': 'ROOT',  'XPOS': 'ROOT',  'FEATS': 'ROOT',  'HEAD': '0',  'DEPREL': 'ROOT',  'DEPS': '0',  'MISC': 'ROOT'}, 
'1': {'ID': '1',  'FORM': 'Les',  'LEMMA': 'le',  'UPOS': 'DET',  'XPOS': '_',  'FEATS': 'Definite=Def|Gender=Masc|Number=Plur|PronType=Art',  'HEAD': '2',  'DEPREL': 'det',  'DEPS': '_',  'MISC': 'wordform=les'}, 
'2': {'ID': '2',  'FORM': 'iris',  'LEMMA': 'iris',  'UPOS': 'NOUN',  'XPOS': '_',  'FEATS': 'Gender=Masc|Number=Plur',  'HEAD': '7',  'DEPREL': 'nsubj',  'DEPS': '_',  'MISC': '_'}, 
'3-4': {'ID': '3-4',  'FORM': 'du',  'LEMMA': '_',  'UPOS': '_',  'XPOS': '_',  'FEATS': '_',  'HEAD': '_',  'DEPREL': '_',  'DEPS': '_',  'MISC': '_'}, 
'3': {'ID': '3',  'FORM': 'de',  'LEMMA': 'de',  'UPOS': 'ADP',  'XPOS': '_',  'FEATS': '_',  'HEAD': '5',  'DEPREL': 'case',  'DEPS': '_',  'MISC': '_'}, 
'4': {'ID': '4',  'FORM': 'le',  'LEMMA': 'le',  'UPOS': 'DET',  'XPOS': '_',  'FEATS': 'Definite=Def|Gender=Masc|Number=Sing|PronType=Art',  'HEAD': '5',  'DEPREL': 'det',  'DEPS': '_',  'MISC': '_'}, 
'5': {'ID': '5',  'FORM': 'mâles',  'LEMMA': 'mâle',  'UPOS': 'NOUN',  'XPOS': '_',  'FEATS': 'Gender=Masc|Number=Plur',  'HEAD': '2',  'DEPREL': 'nmod',  'DEPS': '_',  'MISC': '_'}, 
'6': {'ID': '6',  'FORM': 'sont',  'LEMMA': 'être',  'UPOS': 'AUX',  'XPOS': '_',  'FEATS': 'Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin',  'HEAD': '7',  'DEPREL': 'cop',  'DEPS': '_',  'MISC': '_'}, 
'7': {'ID': '7',  'FORM': 'jaunes',  'LEMMA': 'jaune',  'UPOS': 'ADJ',  'XPOS': '_',  'FEATS': 'Gender=Masc|Number=Plur',  'HEAD': '0',  'DEPREL': 'root',  'DEPS': '_',  'MISC': '_'}, 
'8': {'ID': '8',  'FORM': 'toute',  'LEMMA': 'tout',  'UPOS': 'ADJ',  'XPOS': '_',  'FEATS': 'Gender=Fem|Number=Sing',  'HEAD': '10',  'DEPREL': 'amod',  'DEPS': '_',  'MISC': '_'}, 
'9': {'ID': '9',  'FORM': "l'",  'LEMMA': 'le',  'UPOS': 'DET',  'XPOS': '_',  'FEATS': 'Definite=Def|Gender=Fem|Number=Sing|PronType=Art',  'HEAD': '10',  'DEPREL': 'det',  'DEPS': '_',  'MISC': 'SpaceAfter=No'}, 
'10': {'ID': '10',  'FORM': 'année',  'LEMMA': 'année',  'UPOS': 'NOUN',  'XPOS': '_',  'FEATS': 'Gender=Fem|Number=Sing',  'HEAD': '7',  'DEPREL': 'obl',  'DEPS': '_',  'MISC': 'SpaceAfter=No'}, 
'11': {'ID': '11',  'FORM': '.',  'LEMMA': '.',  'UPOS': 'PUNCT',  'XPOS': '_',  'FEATS': '_',  'HEAD': '7',  'DEPREL': 'punct',  'DEPS': '_',  'MISC': '_'}}
```
3. Some corpora have sentence numbers marked with a `#` symbol. You will ignore them and discard the lines starting with a `#`. This is already done in the CoNLL reader.

#### Extracting the pairs and triples

Write a function `extract_pairs_and_triples(formatted_corpus_dict, nbest)` that extracts the `nbest` most frequent pairs and triples of a given corpus and returns two sorted lists of tuples: `frequent_pairs` and `frequent_triples`. You will sort them by frequency and then by alphabetical order of the pair or triple.

In [26]:
# Write your code here

Run your extractor on all the corpora. Note that some corpora have replaced the words by underscores as for the FTB corpus in French. This is because the text is copyrighted. You need then to contact the provider and sign a license to obtain them.

In [ ]:
# Write your code here

In your report, you will include the `nbest` most frequent pairs and triples you obtained in **three languages**. You may choose the ones you want.

For the checking script, you will extract `nbest` triples in French, Russian, and English. You will rank these triples by frequency, and then by alphabetical order of the triple using `sorted()`. You will use the French GSD corpus, the Russian SynTagRus corpus, and the English EWT corpus. You will store these triples in the following variables:
`freq_triples_fr`, `freq_triples_ru`, `freq_triples_en`. Each variable will contain a list of tuples: `(subject, verb, object), freq)`

Here is what you should find:

French
```
freq_triples_fr = [(('il', 'fait', 'partie'), 16),

 ...]
 ````

And the frequencies:
```
16
7
7
```

Russian:
```
freq_triples_ru = [(('мы', 'имеем', 'дело'), 6),

 ...]
 ````

And the frequencies:
```
6
4
4
```

English:
```
freq_triples_en = [(('you', 'have', 'questions'), 22),

 ...]
 ````

And the frequencies:
```
22
12
7
```

In [28]:
files = [path_fr, path_ru, path_en]

##### French

In [29]:
# Write your code here

In [ ]:
freq_pairs_fr, freq_triples_fr = extract_pairs_and_triples(formatted_corpus_dict, nbest)
freq_triples_fr

##### Russian

In [31]:
# Write your code here

In [ ]:
freq_pairs_ru, freq_triples_ru = extract_pairs_and_triples(formatted_corpus_dict, nbest)
freq_triples_ru

##### English

In [33]:
# Write your code here

In [ ]:
freq_pairs_en, freq_triples_en = extract_pairs_and_triples(formatted_corpus_dict, nbest)
freq_triples_en

## Resolving the entities

You will now extract the relations involving named entities, that is where both the subject and the object are proper nouns. These proper nouns have a specific part of speech in the `UPOS` column.

Write an `extract_entity_triples(formatted_corpus_dict)` that will process the corpus and return a list of `(subject, verb, object)` triples. You will leave the case as it is in the `FORM` column, for instance _United States_ and not _united states_.  

In [35]:
# Write your code here

You will run the `extract_entity_triples()` function one the English EWT corpus. You will store the list in the `entity_relation_en` variable and you will sort it with `sorted()`. You will keep the **five** first triples. 

In [36]:
nbest = 5

The two first triples are:
```
[('Baba', 'remember', 'George'),
 ('Beschta', 'told', 'Planet'),
...]
 ```
Note that this time, we keep the original case and the triples are in the alphabetical order.

In [37]:
#Write your code here

In [ ]:
entity_relation_en = sorted(entity_relation_en)[:nbest]
entity_relation_en

### Extracting chunks

Extracting only the headword of the subject and object if often incomplete and uninformative. You can extract all the chunk instead. 

As an exercise, you will extract manually from the corpus the complete proper nouns in this triple `('Baba', 'remember', 'George')`. You will first consider `Baba` and `George`. You will then look at the verb `remember`.

**In your report**, you will give the complete value of the two proper nouns as well as the verb: `(Complete value of the subject, Complete value of the verb, Complete value of the object)`

### Optional exercise: Automatic Extraction of the chunks

As an optional exercise, you can implement an automatic baseline technique and extract adjacent proper nouns. You may also want to apply the chunker of the 4th assignment to the corpus to do this.

### Optional exercise: Mapping the entities

As in the chunker assignment, you may also want to complement your assignment with a entity solver that will link the entities to wikidata.

## Reading

Read the article: _PRISMATIC: Inducing Knowledge from a Large Scale Lexicalized Relation Resource_ by Fan and al. (2010) [<a href="http://www.aclweb.org/anthology/W/W10/W10-0915.pdf">pdf</a>] and write in a few sentences how it relates to your work in this assignment.

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [42]:
STIL_ID = ["student_1", "student_2"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "5-triples_solution.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the pairs and triples in four languages, as well as the triples with named entities.

In [ ]:
import json
ANSWER = json.dumps({'freq_triples_sv': freq_triples_sv,
                     'freq_triples_fr': freq_triples_fr,
                     'freq_triples_ru': freq_triples_ru,
                     'freq_triples_en': freq_triples_en,
                     'entity_relation_en': entity_relation_en
                    })
ANSWER

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [44]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [45]:
import bz2
ASSIGNMENT = 5
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [46]:
import requests
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
               verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'correct',
 'signature': '7f36e33c7e5de91900cd77a97db905a05dce320e05a2f4b2ad909ec8af41ad9102cdb842f43293bf55dba108005106f373a1a067f06067c3db5616335aae9678',
 'submission_id': '6737f9e7-a9b4-4689-9497-5c8c32624e12'}